In [1]:
from dbconnect.connector import Connection
import pandas as pd
from footmav.data_definitions.whoscored.constants import EventType as EventTypeOld
from footballmodels.opta.event_type import EventType
import json
from footballmodels.opta.actions import ground_duels_total, ground_duels_won, aerial_duels_total, aerial_duels_won, defensive_duel_total, defensive_duel_won
from footmav.utils.whoscored_funcs import minutes, minutes_per_position

In [2]:
conn = Connection('M0neyMa$e')

In [3]:
bl_data = conn.query("SELECT * FROM whoscored T1 LEFT JOIN derived.whoscored_extra_event_info T2 ON T1.id=T2.id WHERE T1.competition='bundesliga' AND T1.season=2023")

In [4]:
b1_data_v1 = bl_data.copy()
b1_data_v1['qualifiers'] = b1_data_v1['qualifiers'].apply(lambda x: json.loads(x) if x else None)
bl_data_v11 = b1_data_v1.copy()
bl_data_v11['event_type'] = bl_data_v11['event_type'].apply(EventTypeOld)
bl_data_v12 = b1_data_v1.copy()
bl_data_v12['event_type'] = bl_data_v12['event_type'].apply(EventType)

In [8]:
bl_data_v12['ground_duels_total'] = ground_duels_total(bl_data_v12).astype(int)
bl_data_v12['ground_duels_won'] = ground_duels_won(bl_data_v12).astype(int)
bl_data_v12['aerial_duels_total'] = aerial_duels_total(bl_data_v12).astype(int)
bl_data_v12['aerial_duels_won'] = aerial_duels_won(bl_data_v12).astype(int)
bl_data_v12['defensive_duel_total'] = defensive_duel_total(bl_data_v12).astype(int)
bl_data_v12['defensive_duel_won'] = defensive_duel_won(bl_data_v12).astype(int)

In [6]:
min_by_pos = minutes_per_position(bl_data_v12)

In [7]:
min_by_pos.reset_index().loc[min_by_pos.reset_index()['player_name']=='ian maatsen']

,matchId,player_name,position,minutes
3921,1743498,ian maatsen,LB,96.950000
4132,1743504,ian maatsen,LB,102.216667
4262,1743508,ian maatsen,LB,86.333333
4263,1743508,ian maatsen,LWB,13.583333
4849,1743524,ian maatsen,LB,96.400000
6622,1743574,ian maatsen,LB,107.333333


In [18]:
agg_data = bl_data_v12.groupby(['player_name','position']).agg(
    {'ground_duels_total': 'sum', 'ground_duels_won': 'sum', 'aerial_duels_total': 'sum', 'aerial_duels_won': 'sum', 'defensive_duel_total': 'sum', 'defensive_duel_won': 'sum'}
).join(min_by_pos.groupby(['player_name','position']).agg({'minutes': 'sum'}))

In [19]:
agg_data = agg_data.reset_index()
agg_data = agg_data.loc[agg_data['position'].isin(['LWB','RWB','LB','RB'])].groupby('player_name').sum()
agg_data

,position,ground_duels_total,ground_duels_won,aerial_duels_total,aerial_duels_won,defensive_duel_total,defensive_duel_won,minutes
player_name,,,,,,,,
aleksandar pavlovic,RB,0,0,0,0,0,0,74.558333
alex grimaldo,LBLWB,115,48,25,7,55,17,1984.966667
alex kral,RWB,6,3,4,1,4,2,122.316667
alphonso davies,LBLWB,176,79,15,10,63,17,1674.741667
amin sarr,LB,0,0,2,0,0,0,50.066667
...,...,...,...,...,...,...,...,...
tuta,RB,20,9,6,3,12,4,209.816667
waldemar anton,RB,5,2,6,5,4,1,163.541667
willi orban,RB,1,0,0,0,1,0,8.158333


In [20]:
agg_data['Ground Duels Per 90']=agg_data['ground_duels_total']/agg_data['minutes']*90
agg_data['Ground Duels Per 90 Rank']=agg_data['Ground Duels Per 90'].rank(ascending=True, method='min', pct=True)

agg_data['Ground Duels Won Per 90']=agg_data['ground_duels_won']/agg_data['minutes']*90
agg_data['Ground Duels Won Per 90 Rank']=agg_data['Ground Duels Won Per 90'].rank(ascending=True, method='min', pct=True)

agg_data['Ground Duels Won %']=agg_data['ground_duels_won']/agg_data['ground_duels_total']
agg_data['Ground Duels Won % Rank']=agg_data['Ground Duels Won %'].rank(ascending=False, method='min', pct=True)


agg_data['Defensive Duels Per 90']=agg_data['defensive_duel_total']/agg_data['minutes']*90
agg_data['Defensive Duels Per 90 Rank']=agg_data['Defensive Duels Per 90'].rank(ascending=True, method='min', pct=True)

agg_data['Defensive Duels Won Per 90']=agg_data['defensive_duel_won']/agg_data['minutes']*90
agg_data['Defensive Duels Won Per 90 Rank']=agg_data['Defensive Duels Won Per 90'].rank(ascending=True, method='min', pct=True)

agg_data['Defensive Duels Won %']=agg_data['defensive_duel_won']/agg_data['defensive_duel_total']
agg_data['Defensive Duels Won % Rank']=agg_data['Defensive Duels Won %'].rank(ascending=False, method='min', pct=True)

agg_data['Aerial Duels Per 90']=agg_data['aerial_duels_total']/agg_data['minutes']*90
agg_data['Aerial Duels Per 90 Rank']=agg_data['Aerial Duels Per 90'].rank(ascending=True, method='min', pct=True)
agg_data['Aerial Duels Won Per 90']=agg_data['aerial_duels_won']/agg_data['minutes']*90
agg_data['Aerial Duels Won Per 90 Rank']=agg_data['Aerial Duels Won Per 90'].rank(ascending=True, method='min', pct=True)
agg_data['Aerial Duels Won %']=agg_data['aerial_duels_won']/agg_data['aerial_duels_total']
agg_data['Aerial Duels Won % Rank']=agg_data['Aerial Duels Won %'].rank(ascending=False, method='min', pct=True)

agg_data['Duels Per 90']=(agg_data['ground_duels_total']+agg_data['aerial_duels_total'])/agg_data['minutes']*90
agg_data['Duels Per 90 Rank']=agg_data['Duels Per 90'].rank(ascending=True, method='min', pct=True)

agg_data['Duels Won Per 90']=(agg_data['ground_duels_won']+agg_data['aerial_duels_won'])/agg_data['minutes']*90
agg_data['Duels Won Per 90 Rank']=agg_data['Duels Won Per 90'].rank(ascending=True, method='min', pct=True)
agg_data['Duels Won %']=(agg_data['ground_duels_won']+agg_data['aerial_duels_won'])/(agg_data['ground_duels_total']+agg_data['aerial_duels_total'])
agg_data['Duels Won % Rank']=agg_data['Duels Won %'].rank(ascending=False, method='min', pct=True)
agg_data = agg_data.drop(columns=['ground_duels_total','ground_duels_won','aerial_duels_total','aerial_duels_won','defensive_duel_total','defensive_duel_won'])


In [30]:
agg_data.loc['ian maatsen'].to_frame().style.format(precision=2)

,ian maatsen
position,LBLWB
minutes,502.82
Ground Duels Per 90,7.52
Ground Duels Per 90 Rank,0.87
Ground Duels Won Per 90,3.40
Ground Duels Won Per 90 Rank,0.88
Ground Duels Won %,0.45
Ground Duels Won % Rank,0.36
Defensive Duels Per 90,3.58
Defensive Duels Per 90 Rank,0.81
